<a href="https://colab.research.google.com/github/MeghanaKankanala/CS-lab20/blob/master/Text_Summerization%20using%20tf_idf%20Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tf_idf Algo**

In [122]:
import nltk
nltk.download('punkt') 
nltk.download('wordnet') 
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Term frequency (TF) is how often a word appears in a document, divided by how many words there are.
# **TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)**
Term frequency is how common a word is, inverse document frequency (IDF) is how unique or rare a word is.
# **IDF(t) = log_e(Total number of documents / Number of documents with term t in it)**
Example,
Consider a document containing 100 words wherein the word apple appears 5 times. The term frequency (i.e., TF) for apple is then (5 / 100) = 0.05.
Now, assume we have 10 million documents and the word apple appears in one thousand of these. Then, the inverse document frequency (i.e., IDF) is calculated as log(10,000,000 / 1,000) = 4.
Thus, the TF-IDF weight is the product of these quantities: 0.05 * 4 = 0.20.

In [123]:
text="To help the actors prepare for their roles, director James Cameron took the cast and crew to Hawaii, where they spent their days trekking through the forests and jungles and living like tribes (building campfires, eating fish, etc), in order to get a better sense of what it would be like to live and move around in the jungle on Pandora, since there would not be any actual jungle sets to aid and guide the actors and crew. Zoe Saldana even dressed up as a warrior during these journeys, complete with an alien tail symbolic of the one her character has in the movie. These hikes were only done during the daytime, however, as the cast and crew spent their nights at a Four Seasons hotel."

# Tokenizing into sentences

In [124]:
sentences = sent_tokenize(text) 
total_documents = len(sentences)
sentences

['To help the actors prepare for their roles, director James Cameron took the cast and crew to Hawaii, where they spent their days trekking through the forests and jungles and living like tribes (building campfires, eating fish, etc), in order to get a better sense of what it would be like to live and move around in the jungle on Pandora, since there would not be any actual jungle sets to aid and guide the actors and crew.',
 'Zoe Saldana even dressed up as a warrior during these journeys, complete with an alien tail symbolic of the one her character has in the movie.',
 'These hikes were only done during the daytime, however, as the cast and crew spent their nights at a Four Seasons hotel.']

Creating freq_matrix for a word within sentences

In [125]:
def frequency_matrix(sentences):
    freq_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        freq_matrix[sent[:10]] = freq_table

    return freq_matrix

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)

In [126]:
def term_frequency_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        no_of_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / no_of_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

defining function which gives no of documents contain the word

In [127]:
def documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

IDF(t) = log_e(Total number of documents / Number of documents with term t in it) 

In [128]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [129]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


sentence scoring
 adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    

In [130]:
def _score_sentences(tf_idf_matrix): 
    

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        total_no_of_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / total_no_of_words_in_sentence

    return sentenceValue

Finding Threshold

In [131]:
def _average_score(sentenceValue):
    
    
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    average = (sumValues / len(sentenceValue))

    return average

In [132]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary


In [133]:
import math
# 2 Create the Frequency matrix of the words in each sentence.
freq_matrix = frequency_matrix(sentences)
print(freq_matrix)





{'To help th': {'help': 1, 'actor': 2, 'prepar': 1, 'role': 1, ',': 6, 'director': 1, 'jame': 1, 'cameron': 1, 'took': 1, 'cast': 1, 'crew': 2, 'hawaii': 1, 'spent': 1, 'day': 1, 'trek': 1, 'forest': 1, 'jungl': 3, 'live': 2, 'like': 2, 'tribe': 1, '(': 1, 'build': 1, 'campfir': 1, 'eat': 1, 'fish': 1, 'etc': 1, ')': 1, 'order': 1, 'get': 1, 'better': 1, 'sens': 1, 'would': 2, 'move': 1, 'around': 1, 'pandora': 1, 'sinc': 1, 'ani': 1, 'actual': 1, 'set': 1, 'aid': 1, 'guid': 1, '.': 1}, 'Zoe Saldan': {'zoe': 1, 'saldana': 1, 'even': 1, 'dress': 1, 'warrior': 1, 'dure': 1, 'journey': 1, ',': 1, 'complet': 1, 'alien': 1, 'tail': 1, 'symbol': 1, 'one': 1, 'charact': 1, 'ha': 1, 'movi': 1, '.': 1}, 'These hike': {'hike': 1, 'onli': 1, 'done': 1, 'dure': 1, 'daytim': 1, ',': 2, 'howev': 1, 'cast': 1, 'crew': 1, 'spent': 1, 'night': 1, 'four': 1, 'season': 1, 'hotel': 1, '.': 1}}


In [134]:
# 3 Calculate TermFrequency and generate a matrix
tf_matrix = term_frequency_matrix(freq_matrix)
print(tf_matrix)



{'To help th': {'help': 0.023809523809523808, 'actor': 0.047619047619047616, 'prepar': 0.023809523809523808, 'role': 0.023809523809523808, ',': 0.14285714285714285, 'director': 0.023809523809523808, 'jame': 0.023809523809523808, 'cameron': 0.023809523809523808, 'took': 0.023809523809523808, 'cast': 0.023809523809523808, 'crew': 0.047619047619047616, 'hawaii': 0.023809523809523808, 'spent': 0.023809523809523808, 'day': 0.023809523809523808, 'trek': 0.023809523809523808, 'forest': 0.023809523809523808, 'jungl': 0.07142857142857142, 'live': 0.047619047619047616, 'like': 0.047619047619047616, 'tribe': 0.023809523809523808, '(': 0.023809523809523808, 'build': 0.023809523809523808, 'campfir': 0.023809523809523808, 'eat': 0.023809523809523808, 'fish': 0.023809523809523808, 'etc': 0.023809523809523808, ')': 0.023809523809523808, 'order': 0.023809523809523808, 'get': 0.023809523809523808, 'better': 0.023809523809523808, 'sens': 0.023809523809523808, 'would': 0.047619047619047616, 'move': 0.0238

In [135]:
# 4 creating table for documents per words
count_doc_per_words = documents_per_words(freq_matrix)
print(count_doc_per_words)



{'help': 1, 'actor': 1, 'prepar': 1, 'role': 1, ',': 3, 'director': 1, 'jame': 1, 'cameron': 1, 'took': 1, 'cast': 2, 'crew': 2, 'hawaii': 1, 'spent': 2, 'day': 1, 'trek': 1, 'forest': 1, 'jungl': 1, 'live': 1, 'like': 1, 'tribe': 1, '(': 1, 'build': 1, 'campfir': 1, 'eat': 1, 'fish': 1, 'etc': 1, ')': 1, 'order': 1, 'get': 1, 'better': 1, 'sens': 1, 'would': 1, 'move': 1, 'around': 1, 'pandora': 1, 'sinc': 1, 'ani': 1, 'actual': 1, 'set': 1, 'aid': 1, 'guid': 1, '.': 3, 'zoe': 1, 'saldana': 1, 'even': 1, 'dress': 1, 'warrior': 1, 'dure': 2, 'journey': 1, 'complet': 1, 'alien': 1, 'tail': 1, 'symbol': 1, 'one': 1, 'charact': 1, 'ha': 1, 'movi': 1, 'hike': 1, 'onli': 1, 'done': 1, 'daytim': 1, 'howev': 1, 'night': 1, 'four': 1, 'season': 1, 'hotel': 1}


In [136]:
# 5 Calculate IDF and generate a matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
print(idf_matrix)



{'To help th': {'help': 0.47712125471966244, 'actor': 0.47712125471966244, 'prepar': 0.47712125471966244, 'role': 0.47712125471966244, ',': 0.0, 'director': 0.47712125471966244, 'jame': 0.47712125471966244, 'cameron': 0.47712125471966244, 'took': 0.47712125471966244, 'cast': 0.17609125905568124, 'crew': 0.17609125905568124, 'hawaii': 0.47712125471966244, 'spent': 0.17609125905568124, 'day': 0.47712125471966244, 'trek': 0.47712125471966244, 'forest': 0.47712125471966244, 'jungl': 0.47712125471966244, 'live': 0.47712125471966244, 'like': 0.47712125471966244, 'tribe': 0.47712125471966244, '(': 0.47712125471966244, 'build': 0.47712125471966244, 'campfir': 0.47712125471966244, 'eat': 0.47712125471966244, 'fish': 0.47712125471966244, 'etc': 0.47712125471966244, ')': 0.47712125471966244, 'order': 0.47712125471966244, 'get': 0.47712125471966244, 'better': 0.47712125471966244, 'sens': 0.47712125471966244, 'would': 0.47712125471966244, 'move': 0.47712125471966244, 'around': 0.47712125471966244, 

In [137]:
# 6 Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
print(tf_idf_matrix)



{'To help th': {'help': 0.011360029874277677, 'actor': 0.022720059748555354, 'prepar': 0.011360029874277677, 'role': 0.011360029874277677, ',': 0.0, 'director': 0.011360029874277677, 'jame': 0.011360029874277677, 'cameron': 0.011360029874277677, 'took': 0.011360029874277677, 'cast': 0.004192649025135268, 'crew': 0.008385298050270535, 'hawaii': 0.011360029874277677, 'spent': 0.004192649025135268, 'day': 0.011360029874277677, 'trek': 0.011360029874277677, 'forest': 0.011360029874277677, 'jungl': 0.03408008962283303, 'live': 0.022720059748555354, 'like': 0.022720059748555354, 'tribe': 0.011360029874277677, '(': 0.011360029874277677, 'build': 0.011360029874277677, 'campfir': 0.011360029874277677, 'eat': 0.011360029874277677, 'fish': 0.011360029874277677, 'etc': 0.011360029874277677, ')': 0.011360029874277677, 'order': 0.011360029874277677, 'get': 0.011360029874277677, 'better': 0.011360029874277677, 'sens': 0.011360029874277677, 'would': 0.022720059748555354, 'move': 0.011360029874277677, 

In [138]:
# 7 Important Algorithm: score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
print(sentence_scores)



{'To help th': 0.012029806683201942, 'Zoe Saldan': 0.023722452682114033, 'These hike': 0.022215361460887494}


In [139]:
# 8 Find the threshold
threshold = _average_score(sentence_scores)
print(threshold)

0.019322540275401156


In [140]:
# 9 Important Algorithm: Generate the summary
summary = _generate_summary(sentences,sentence_scores,threshold)
print(summary)

 Zoe Saldana even dressed up as a warrior during these journeys, complete with an alien tail symbolic of the one her character has in the movie. These hikes were only done during the daytime, however, as the cast and crew spent their nights at a Four Seasons hotel.
